Напишем генератор задач по линейной алгебре по следующему прототипу: <br>
"Дано комплексное число $z = a + b \cdot i$. Найти $k$-ый по счету корень $n$-ой степени из $z$, начиная от угла в $0$ градусов в сторону повышения угла поворота"

Сначала подключаем все необходимые библиотеки

In [1]:
import random
import sympy
from sympy import I, latex
from IPython.display import display, Latex

Напишем функцию, которая генерирует $2$ случайных вещественных числа ($a$ и  $b$ соответственно), а также функция будет генерировать $2$ натуральных числа ($n$ и $k$ соответсвенно). Функция возврщается картеж из сгенерированных значений.

In [2]:
def generate_random_int(
    min_value:int=1,
    max_value:int=10     
) -> int:
    return random.randint(min_value, max_value)

In [3]:
def generate_random_float(
    min_value:float=1.0,
    max_value:float=10.0,
    digits:int=2
) -> float:
    return round(random.uniform(min_value, max_value), digits)

In [4]:
def generate_random_numbers(
    to_generate:dict[str: tuple[str, str, bool]]
) -> dict[str: tuple[float, bool]]:
    result = dict()
    for num, data in to_generate.items():
        left_border, right_border = 0.0, 0.0

        if data[0].isalpha():
            if not data[0] in result.keys():
                raise ValueError(f"Variable {data[0]} is not defined")
            left_border = result[data[0]][0]
        elif data[0].isnumeric():
            left_border = float(data[0])
        else:
            raise ValueError(f"Invalid left border value: {data[0]}")


        if data[1].isalpha():
            if not data[1] in result.keys():
                raise ValueError(f"Variable {data[1]} is not defined")
            right_border = result[data[1]][0]
        elif data[1].isnumeric():
            right_border = float(data[1])
        else:
            raise ValueError(f"Invalid right border value: {data[1]}")
        
        if data[2]:
            result[num] = (generate_random_int(int(left_border), int(right_border)), data[2])
        else:
            result[num] = (generate_random_float(left_border, right_border), data[2])

    return result
        

Для создания $LaTeX$ файла необходимы заголовочные файлы, поэтому запишем все потенциально необходимые подключеия в отдельную переменную. 

In [5]:
default_headers = r"""
\documentclass[12pt]{extarticle}
\usepackage{graphicx}
\usepackage{amsmath}
\usepackage{mathabx}
\usepackage[utf8]{inputenc}
\usepackage[T1,T2A]{fontenc}
\usepackage[russian]{babel}
\usepackage{amsfonts}
\usepackage{gensymb}
\usepackage{listings}
\usepackage{titlesec}
\usepackage{color}
\usepackage[dvipsnames]{xcolor}
\usepackage{soul}
\usepackage{xfrac}
\usepackage{hyperref}
\usepackage[margin=0.5in]{geometry}
\usepackage{tikz}
\usetikzlibrary{decorations.pathreplacing,calligraphy}

\usepackage{blindtext}

\newsavebox\MBox
\newcommand\Cline[2][red]{{\sbox\MBox{$#2$}%
  \rlap{\usebox\MBox}\color{#1}\rule[-1.2\dp\MBox]{\wd\MBox}{0.5pt}}}

\newcommand{\mgcd}[2]{{\text{GCD}\left(#1,\, #2\right)}}
\newcommand{\fib}[1]{\operatorname{Fib}_{#1}}
\newcommand{\mred}[1]{{\color{red}#1}}
\newcommand{\mpurple}[1]{{\color{purple}#1}}
\newcommand{\morange}[1]{{\color{orange}#1}}
\newcommand{\mblue}[1]{{\color{blue}#1}}
\newcommand{\mcol}[2]{{\color{#1}#2}}
\newcommand{\mprime}[1]{#1^{\prime}}
\newcommand{\highlight}[2][yellow]{\mathchoice%
  {\colorbox{#1}{$\displaystyle#2$}}%
  {\colorbox{#1}{$\textstyle#2$}}%
  {\colorbox{#1}{$\scriptstyle#2$}}%
  {\colorbox{#1}{$\scriptscriptstyle#2$}}}%

\newcommand{\myul}[2][black]{\setulcolor{#1}\ul{#2}\setulcolor{black}}

% \titleformat{\subsection}[block]{\color{black}\Large\bfseries\filcenter}{}{1em}{}

\definecolor{dkgreen}{rgb}{0,0.6,0}
\definecolor{gray}{rgb}{0.5,0.5,0.5}
\definecolor{mauve}{rgb}{0.58,0,0.82}

% \textwidth 20cm

\lstset{frame=tb,
  language=Python,
  aboveskip=3mm,
  belowskip=3mm,
  showstringspaces=false,
  columns=flexible,
  basicstyle={\small\ttfamily},
  numbers=none,
  numberstyle=\tiny\color{gray},
  keywordstyle=\color{blue},
  commentstyle=\color{dkgreen},
  stringstyle=\color{mauve},
  breaklines=true,
  breakatwhitespace=true,
  tabsize=3
}
"""

Напишем функцию, которая будет создавать $LaTeX$ код при помощи переданных в него заголовков и сгенерированных чисел.

In [6]:
def generate_problem_latex(
        original_text:str,
        numbers:dict[str: tuple[float, bool]],
        headers:str=default_headers
) -> str:
        problem = headers
        problem += r"\begin{document}"
        problem += original_text
        problem += r"$$"
        for number, data in numbers.items():
                problem += number
                problem += r" = "
                if data[1]:
                        problem += str(int(data[0]))
                else:
                        problem += str(data[0])
                if number != list(numbers.keys())[-1]:
                        problem += r",\ "
        problem += r"$$"
        problem += r"\end{document}"
        return problem

Создадим функцию, которая сохраняет созданный код в $.tex$ файл по указанному пути

In [7]:
def write_code_to_file(
        code:str, 
        pathname:str="problem.tex"
) -> None:
    if len(pathname) < 4:
        raise ValueError("File name must have at least 4 characters")
    if pathname[-4:] != ".tex":
        raise ValueError("File must have a .tex extension")
    with open(pathname, 'w') as file:
        file.write(code)

Протестируем код

In [10]:
prob = generate_problem_latex(
    """sosal?""",
    generate_random_numbers(
        {"a": ("1", "10", True), "b": ("a", "20", False), "c": ("b", "b", False)}
    )
)

write_code_to_file(prob, "build/problem.tex")